# Zero-Shot Prompting

Zero-shot prompting refers to a technique in prompt engineering where you provide a model with a task without any prior examples. The model is expected to understand and generate a response or complete the task purely based on the given instruction.

In other words, the model is given "zero" prior training examples or demonstrations in the prompt and relies on its pre-trained knowledge to infer what is needed.

## References:
* [Wei et al. (2022)](https://arxiv.org/pdf/2109.01652.pdf): demonstrate how instruction tuning improves zero-shot learning 

## Running this code on MyBind.org

Note: remember that you will need to **adjust CONFIG** with **proper URL and API_KEY**!

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/GenILab-FAU/prompt-eng/HEAD?urlpath=%2Fdoc%2Ftree%2Fprompt-eng%2Fzero_shot.ipynb)



In [10]:
############################################################
## ZERO-SHOT PROMPT: PROJECT OVERVIEW
############################################################
import os

os.makedirs("data", exist_ok=True)

import csv
from datetime import datetime
from _pipeline import create_payload, model_req

# 1) Define the zero-shot prompt (no examples)
ZERO_SHOT_PROMPT = """You are an AI that generates project reports.
Please create the "Project Overview" section covering:
- Title
- Goal
- Problem Statement
- Key Objectives
- Scope
Format it in clear paragraphs, and use Markdown headings.
"""

# 2) Prepare the payload (choose model & parameters)
payload = create_payload(
    target="open-webui",
    model="Llama-3.2-3B-Instruct",
    prompt=ZERO_SHOT_PROMPT,
    temperature=0.9,      # Slightly higher => more creative language
    max_tokens=300,       # Adjust if you see truncated output
    num_ctx=100,          # Enough context for a short prompt
    num_predict=200       # Sufficient tokens for a robust overview
)

# 3) Send request and capture the response
time_taken, zero_shot_response = model_req(payload=payload)

# 4) Display the raw output and timing
print("===== ZERO-SHOT OUTPUT =====")
print(zero_shot_response)
if time_taken:
    print(f"\nTime taken: {time_taken}s")

# 5) (Optional) Log the result for future reference
#    This helps if you want to compare with Few-Shot or Chain-of-Thought later.
log_entry = [
    datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "zero_shot",
    "Llama-3.2-3B-Instruct",
    0.9,
    time_taken,
    zero_shot_response.replace("\n", "\\n")  # escape newlines for CSV
]

with open("data/zero_shot_logs.csv", "a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    # If it's a brand-new file, you could write a header row here
    writer.writerow(log_entry)


Payload: {'model': 'Llama-3.2-3B-Instruct', 'messages': [{'role': 'user', 'content': 'You are an AI that generates project reports.\nPlease create the "Project Overview" section covering:\n- Title\n- Goal\n- Problem Statement\n- Key Objectives\n- Scope\nFormat it in clear paragraphs, and use Markdown headings.\n'}]}
===== ZERO-SHOT OUTPUT =====
**Project Overview**

### Title

**E-commerce Platform Development**

### Goal

Our goal is to design and develop a modern, user-friendly, and scalable e-commerce platform that enables businesses to create engaging online shopping experiences, process transactions efficiently, and analyze customer behavior effectively.

### Problem Statement

The existing e-commerce platforms face several challenges, including high transaction fees, limited customization options, poor user experience, and inadequate data security. These issues hinder the growth and success of online businesses, making it difficult for them to compete in the market.

### Key Obje

---

## How to improve it?

* **Use Clear and Concise Instructions**: Be specific about the task and desired format.
    * Bad Prompt: “Summarize this.”
    * Good Prompt: “Summarize this paragraph in one sentence.”
* **Add Context**: Providing background can help the model interpret ambiguous prompts better.
* **Specify Output Format**: If a particular structure is needed, describe it in the instruction.